### Encoder


In [ ]:
import os
import sys


def add_to_path(new_path: str):
    module_path = os.path.abspath(os.path.join(new_path))
    if module_path not in sys.path:
        sys.path.append(module_path)


is_colab = False

if is_colab:
    !git clone https://github.com/pdkary/Karys.git
    !cd Karys && git fetch && git pull
    !cd Karys && pip install -r requirements.txt --quiet
    add_to_path('Karys/')
    from google.colab import drive
    drive.mount('/content/drive')
    !cd Karys && pip install -r requirements.txt --quiet
else:
    add_to_path('../../')
    !cd ../../ && pip install -r requirements.txt --quiet


#### Load the data

In [ ]:
from data.configs.ImageDataConfig import ImageDataConfig
from data.wrappers.ImageDataWrapper import ImageDataWrapper
import numpy as np

if is_colab:
    base_path = "drive/MyDrive/Colab/Seefood/Fruit"
else:
    base_path = "../discriminator/test_input/Fruit"

image_config = ImageDataConfig(image_shape=(256,256,3),image_type=".jpg", preview_rows=4, preview_cols=2, load_n_percent=8)

data_wrapper = ImageDataWrapper.load_from_labelled_directories(base_path + '/', image_config, validation_percentage=0.1)
classification_labels = list(set(data_wrapper.image_labels.values()))

##Quick stats about loaded images
print("----------LOADED IMAGE CATEGORIES----------")
print(classification_labels)
print("NUM IMAGES: ",len(data_wrapper.image_labels))

print("----------LOADED IMAGE STATS----------")
value_count_dict = {v:0 for v in set(data_wrapper.image_labels.values())}
for x in data_wrapper.image_labels.values():
    value_count_dict[x]+=1
print(value_count_dict)
print("Max: ",np.max(list(value_count_dict.values())), max(value_count_dict,key=value_count_dict.get))
print("Min: ",np.min(list(value_count_dict.values())), min(value_count_dict,key=value_count_dict.get))
print("Mean: ",np.mean(list(value_count_dict.values())))
print("Median: ",np.median(list(value_count_dict.values())))
print("Deviation: ",np.std(list(value_count_dict.values())))

### We want an encoder, and a classifier to identify the encodings

In [ ]:
from data.saved_models.SavedModelService import SavedModelService
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy, MSE, CategoricalCrossentropy

load_from_saved_models = False

if is_colab:
  model_reference_path = "drive/MyDrive/Colab/Seefood/models/model_ref.json"
else:
  model_reference_path = "./test_model_output/model_ref.json"

saved_model_service = SavedModelService(model_reference_path)

## Encoded Classifier

In [ ]:
from models.EncoderModel import EncoderModel

from tensorflow.keras.layers import Conv2D, Dense, LeakyReLU, MaxPooling2D, Flatten, Activation, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy, Reduction

ENCODED_DIM = 512
a = 0.08

encoder_layers = [
    Conv2D(128,3), BatchNormalization(), LeakyReLU(a),
    Conv2D(128,3), BatchNormalization(), LeakyReLU(a),
    MaxPooling2D(),
    Conv2D(256,3), BatchNormalization(), LeakyReLU(a),
    Conv2D(256,3), BatchNormalization(), LeakyReLU(a),
    Conv2D(256,3), BatchNormalization(), LeakyReLU(a),
    MaxPooling2D(),
    Conv2D(512,3), BatchNormalization(), LeakyReLU(a),
    Conv2D(512,3), BatchNormalization(), LeakyReLU(a),
    Conv2D(512,3), BatchNormalization(), LeakyReLU(a),
    MaxPooling2D(),
    Flatten(),
    Dense(4096), Activation('tanh'),
    Dense(ENCODED_DIM), Activation('sigmoid'),
]

encoder_optimizer = Adam(learning_rate=5e-5, decay=1e-9)
encoder_loss = CategoricalCrossentropy(reduction=Reduction.SUM)

encoder = EncoderModel(image_config.image_shape, ENCODED_DIM, encoder_layers, encoder_optimizer, encoder_loss)

In [ ]:
from models.ClassificationModel import ClassificationModel

classifier_optimizer = Adam(learning_rate=5e-5)
classifier_loss = CategoricalCrossentropy(reduction=Reduction.SUM)

classifier_layers = [
    Dense(ENCODED_DIM), Activation('tanh'),
    Dense(ENCODED_DIM), Activation('tanh'),
    Dense(ENCODED_DIM), Activation('tanh'),
    Dense(len(classification_labels)+1), Activation('softmax'),
]

classifier = ClassificationModel(image_config.image_shape, classification_labels, ['invalid'], classifier_layers, classifier_optimizer, classifier_loss)

In [ ]:
from models.EncodedClassificationModel import EncodedClassificationModel

encoded_classifier = EncodedClassificationModel(encoder, classifier)
encoded_classifier.build()

## Training Loop

In [ ]:
from plotting.TrainPlotter import TrainPlotter
from trainers.EncoderTrainer import EncoderTrainer

epochs=2000
trains_per_test=250

train_columns = ["Encoder Loss", "Encoded Classifier Loss", "Image Classifier Loss", "Test Encoder Loss", "Test Encoded Classifier Loss", "Test Image Classifier Loss"]
loss_plot = TrainPlotter(moving_average_size=100, labels=train_columns)
trainer = EncoderTrainer(encoded_classifier, data_wrapper)

if is_colab:
  image_output_path = "drive/MyDrive/Colab/Seefood/output"
else:
  image_output_path = "./test_output"

e_test_loss, ec_test_loss, ic_test_loss = 0, 0, 0
for i in range(epochs):
  loss_plot.start_epoch()  
  e_loss, ec_loss, ic_loss = trainer.train(5, 1)

  if i % trains_per_test == 0 and i != 0:
    e_test_loss, ec_test_loss, ic_test_loss = trainer.test(16,1)
    encoding_output_filename = image_output_path + "/encoded-" + str(i) + ".jpg"
    classification_output_filename = image_output_path + "/classified-" + str(i) + ".jpg"

    data_wrapper.save_encoded_images(encoding_output_filename, trainer.most_recent_encoding, img_size=32)
    data_wrapper.save_classified_images(classification_output_filename, trainer.most_recent_classification, img_size=32)
      
  loss_plot.batch_update([e_loss, ec_loss, ic_loss, e_test_loss, ec_test_loss, ic_test_loss])
  loss_plot.log_epoch()

## Now we save the models

In [ ]:
from data.saved_models.SavedModelService import SavedModelService

if is_colab:
  model_output_path = "drive/MyDrive/Colab/Seefood/models"
  model_reference_path = "drive/MyDrive/Colab/Seefood/models/model_ref.json"
else:
  model_output_path = "./test_model_output"
  model_reference_path = "./test_model_output/model_ref.json"

saved_model_service = SavedModelService(model_reference_path)
saved_model_service.save_model("FruitEncoder", model_output_path, encoded_classifier.encoder)
saved_model_service.save_model("EncodedFruitClassifier", model_output_path, encoded_classifier.classifier)